In [39]:
import pyspark
from pyspark.sql import SparkSession

import pandas as pd
from pyspark.sql import types
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 16:47:42 WARN Utils: Your hostname, codespaces-eb9fcb resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/03 16:47:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 16:47:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [43]:
df = spark.read.parquet('fhvhv/2019/10/')

In [44]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   null|                  null|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   null|                B02429|
|              B01482|2019-10-19 12:00:00|2019-10-19 12:20:00|         264|         264|   null|                B01482|
|              B03015|2019-10-11 14:28:00|2019-10-11 14:32:44|         264|         216|   null|                B03015|
|              B01529|2019-10-21 18:00:26|2019-10-21 18:07:21|         264|          80|   null|                B01529|
|              B00477|2019-10-03 19:30:3

In [45]:
df.registerTempTable('trips')

/home/codespace/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [46]:
# Q3 - How many taxi trips were there on the 15th of October?
spark.sql("""
SELECT 
    COUNT(*) qty
FROM  
    trips
WHERE 
    pickup_datetime BETWEEN '2019-10-15 00:00:00' and '2019-10-15 23:59:59'
""").show()

+-----+
|  qty|
+-----+
|62610|
+-----+



In [47]:
# Q4 - What is the length of the longest trip in the dataset in hours?
spark.sql("""
SELECT 
  MAX(datediff(dropoff_datetime,pickup_datetime) * 24) max_trip_hour
FROM
    trips
""").show()

+-------------+
|max_trip_hour|
+-------------+
|       631152|
+-------------+



In [48]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
])

df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv('taxi+_zone_lookup.csv')

df_zones.write.parquet('zones', mode='overwrite')
# df_zones.show()


In [49]:
df_zones.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [50]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [51]:
# Q6 - Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?
df_result = df.withColumn('LocationID', col('PULocationID')) \
    .join(df_zones, on='LocationID')

In [52]:
df_result.write.parquet('reports/data')

In [54]:
df_result.registerTempTable('trips_and_zones')

In [58]:
spark.sql("""
SELECT
    LocationID,
    Zone,
    COUNT(1) as qty
FROM 
    trips_and_zones
GROUP BY 
    LocationID, Zone
ORDER BY 3 ASC
""").show()

+----------+--------------------+---+
|LocationID|                Zone|qty|
+----------+--------------------+---+
|         2|         Jamaica Bay|  1|
|       105|Governor's Island...|  2|
|       111| Green-Wood Cemetery|  5|
|        30|       Broad Channel|  8|
|       120|     Highbridge Park| 14|
|        12|        Battery Park| 15|
|       207|Saint Michaels Ce...| 23|
|        27|Breezy Point/Fort...| 25|
|       154|Marine Park/Floyd...| 26|
|         8|        Astoria Park| 29|
|       128|    Inwood Hill Park| 39|
|       253|       Willets Point| 47|
|        96|Forest Park/Highl...| 53|
|        34|  Brooklyn Navy Yard| 57|
|        59|        Crotona Park| 62|
|        58|        Country Club| 77|
|        99|     Freshkills Park| 89|
|       190|       Prospect Park| 98|
|        54|     Columbia Street|105|
|       217|  South Williamsburg|110|
+----------+--------------------+---+
only showing top 20 rows

